<a href="https://colab.research.google.com/github/AI4Finance-LLC/FinRL-Library/blob/master/FinRL_stock_trading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

Tutorials to use OpenAI DRL to trade multiple stocks in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog uses FinRL to reproduce the paper: Practical Deep Reinforcement Learning Approach for Stock Trading, Workshop on Challenges and Opportunities for AI in Financial Services, NeurIPS 2018.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-multiple-stock-trading-7b00763b7530
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [1]:
## install finrl library
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-y67num83
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-y67num83
     |████████████████████████████████| 92kB 5.6MB/s 
     |████████████████████████████████| 153kB 14.9MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 1.4MB 16.7MB/s 
     |████████████████████████████████| 2.0MB 44.2MB/s 
     |████████████████████████████████| 5.5MB 52.9MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 368kB 35.9MB/s 
     |████████████████████████████████| 3.2MB 36.0MB/s 
     |████████████████████████████████| 1.3MB 40.8MB/s 
     |████████████████████████████████| 256kB 48.0MB/s 
     |████████████████████████████████| 235kB 48.9MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 143kB 43.3MB/s 
  Created wheel for 


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/usr/local/lib/python3.6/dist-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


<a id='1.4'></a>
## 2.4. Create Folders

In [3]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [4]:
# from config.py start_date is a string
config.START_DATE

'2000-01-01'

In [5]:
# from config.py end_date is a string
config.END_DATE

'2021-01-01'

In [6]:
print(config.DOW_30_TICKER)

['AAPL', 'MSFT', 'JPM', 'V', 'RTX', 'PG', 'GS', 'NKE', 'DIS', 'AXP', 'HD', 'INTC', 'WMT', 'IBM', 'MRK', 'UNH', 'KO', 'CAT', 'TRV', 'JNJ', 'CVX', 'MCD', 'VZ', 'CSCO', 'XOM', 'BA', 'MMM', 'PFE', 'WBA', 'DD']


In [7]:
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-01-01',
                     ticker_list = config.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
df.shape

(90630, 8)

In [9]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.791740,746015200,AAPL,4
1,2009-01-02,18.570000,19.520000,18.400000,15.745411,10955700,AXP,4
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4
3,2009-01-02,44.910000,46.980000,44.709999,32.978760,7117200,CAT,4
4,2009-01-02,16.410000,17.000000,16.250000,12.683227,40980600,CSCO,4


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [10]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [11]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [12]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.791740,746015200.0,4.0,0.000000,3.017278,2.684025,100.000000,66.666667,100.000000,2.791740,2.791740,0.0
1,2009-01-02,AXP,18.570000,19.520000,18.400000,15.745411,10955700.0,4.0,0.002643,3.017278,2.684025,100.000000,66.666667,100.000000,2.850651,2.850651,0.0
2,2009-01-02,BA,42.799999,45.560001,42.779999,33.941101,7010200.0,4.0,0.001880,2.972787,2.735796,70.355711,46.771878,100.000000,2.854292,2.854292,0.0
3,2009-01-02,CAT,44.910000,46.980000,44.709999,32.978760,7117200.0,4.0,-0.000746,2.951725,2.729582,50.429389,-29.777993,43.607834,2.840654,2.840654,0.0
4,2009-01-02,CSCO,16.410000,17.000000,16.250000,12.683227,40980600.0,4.0,-0.000088,2.939568,2.746169,60.227126,-9.019317,48.357918,2.842869,2.842869,0.0
5,2009-01-02,CVX,74.230003,77.300003,73.580002,48.043262,13695900.0,4.0,-0.002480,2.931445,2.735506,47.932736,-27.237701,49.952380,2.833476,2.833476,0.0
6,2009-01-02,DD,21.605234,22.060680,20.993229,15.402899,13251037.0,4.0,-0.006646,2.938422,2.698235,40.237370,-101.614424,9.320766,2.818329,2.818329,0.0
7,2009-01-02,DIS,22.760000,24.030001,22.500000,20.597496,9796600.0,4.0,-0.010724,2.943226,2.663404,37.181826,-113.576183,3.614243,2.803315,2.803315,0.0
8,2009-01-02,GS,84.019997,87.620003,82.190002,72.844467,14088500.0,4.0,-0.017016,2.960305,2.606701,31.067739,-136.004971,17.746098,2.783503,2.783503,0.0
9,2009-01-02,HD,23.070000,24.190001,22.959999,17.909452,14902500.0,4.0,-0.024322,2.978173,2.545136,27.267345,-166.471239,43.235022,2.761654,2.761654,0.0


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2018-12-31
## Trade data split: 2019-01-01 to 2020-09-30

In [13]:
train = data_split(processed_full, '2009-01-01','2019-01-01')
trade = data_split(processed_full, '2019-01-01','2021-01-01')
print(len(train))
print(len(trade))

75480
15150


In [14]:
train.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.791740,746015200.0,4.0,0.000000,3.017278,2.684025,100.000000,66.666667,100.000000,2.791740,2.791740,0.0
0,2009-01-02,AXP,18.570000,19.520000,18.400000,15.745411,10955700.0,4.0,0.002643,3.017278,2.684025,100.000000,66.666667,100.000000,2.850651,2.850651,0.0
0,2009-01-02,BA,42.799999,45.560001,42.779999,33.941101,7010200.0,4.0,0.001880,2.972787,2.735796,70.355711,46.771878,100.000000,2.854292,2.854292,0.0
0,2009-01-02,CAT,44.910000,46.980000,44.709999,32.978760,7117200.0,4.0,-0.000746,2.951725,2.729582,50.429389,-29.777993,43.607834,2.840654,2.840654,0.0
0,2009-01-02,CSCO,16.410000,17.000000,16.250000,12.683227,40980600.0,4.0,-0.000088,2.939568,2.746169,60.227126,-9.019317,48.357918,2.842869,2.842869,0.0


In [15]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2019-01-02,AAPL,38.722500,39.712502,38.557499,38.505024,148158800.0,2.0,2.335191,334.139821,308.258949,47.826590,-27.772146,14.525416,314.202890,312.704339,51.409032
0,2019-01-02,AXP,93.910004,96.269997,93.769997,92.319603,4175400.0,2.0,0.775741,335.611436,305.851699,46.156688,-67.860390,22.313462,314.159711,312.579659,51.409032
0,2019-01-02,BA,316.190002,323.950012,313.709991,314.645142,3292200.0,2.0,-0.338838,336.485023,304.296566,46.806048,-72.067253,24.597730,314.064055,312.452928,51.409032
0,2019-01-02,CAT,124.029999,127.879997,123.000000,118.671188,4783200.0,2.0,-0.655001,336.668731,303.510025,49.792068,-33.729612,14.054906,314.492846,312.392662,51.409032
0,2019-01-02,CSCO,42.279999,43.200001,42.209999,40.057236,23833500.0,2.0,-0.159995,336.703986,304.162298,53.389344,31.217425,7.135987,315.392609,312.451774,51.409032


In [16]:
config.TECHNICAL_INDICATORS_LIST

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [17]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 30, State Space: 301


In [18]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [19]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [20]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [ ]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device


/opt/conda/lib/python3.6/site-packages/torch/cuda/__init__.py:81: UserWarning: 
    Found GPU1 GeForce GTX 760 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=100000)

Logging to tensorboard_log/a2c/a2c_4
------------------------------------
| time/                 |          |
|    fps                | 81       |
|    iterations         | 100      |
|    time_elapsed       | 6        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -42.6    |
|    explained_variance | 0.338    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -41.5    |
|    std                | 1        |
|    value_loss         | 1.22     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 84       |
|    iterations         | 200      |
|    time_elapsed       | 11       |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -42.6    |
|    explained_variance | 0.0336   |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|

KeyboardInterrupt: 

### Model 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

Logging to tensorboard_log/ddpg/ddpg_1
begin_total_asset:1000000
end_total_asset:3761309.8057632465
total_reward:2761309.8057632465
total_cost:  6807.077776350557
total_trades:  39070
Sharpe:  1.0173492167488003
begin_total_asset:1000000
end_total_asset:4423657.61673363
total_reward:3423657.61673363
total_cost:  1277.392035166502
total_trades:  32819
Sharpe:  0.8726982452731067
begin_total_asset:1000000
end_total_asset:4423657.61673363
total_reward:3423657.61673363
total_cost:  1277.392035166502
total_trades:  32819
Sharpe:  0.8726982452731067
begin_total_asset:1000000
end_total_asset:4423657.61673363
total_reward:3423657.61673363
total_cost:  1277.392035166502
total_trades:  32819
Sharpe:  0.8726982452731067
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 38       |
|    time_elapsed    | 258      |
|    total timesteps | 10064    |
| train/             |          |
|    actor_loss      | -2.81    |
|    crit

### Model 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=100000)

Logging to tensorboard_log/ppo/ppo_2
-----------------------------
| time/              |      |
|    fps             | 104  |
|    iterations      | 1    |
|    time_elapsed    | 19   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 102         |
|    iterations           | 2           |
|    time_elapsed         | 39          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014151055 |
|    clip_fraction        | 0.212       |
|    clip_range           | 0.2         |
|    entropy_loss         | -42.6       |
|    explained_variance   | -28.1       |
|    learning_rate        | 0.00025     |
|    loss                 | 5.76        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0277     |
|    std                  | 1           |
|    value_loss           | 12         

### Model 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

Logging to tensorboard_log/td3/td3_2
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 33       |
|    time_elapsed    | 296      |
|    total timesteps | 10064    |
| train/             |          |
|    actor_loss      | 67.9     |
|    critic_loss     | 979      |
|    learning_rate   | 0.001    |
|    n_updates       | 7548     |
---------------------------------
day: 2515, episode: 10
begin_total_asset:1000000.00
end_total_asset:4438572.29
total_reward:3438572.29
total_cost: 1038.05
total_trades: 40290
Sharpe: 1.049
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 30       |
|    time_elapsed    | 669      |
|    total timesteps | 20128    |
| train/             |          |
|    actor_loss      | 54       |
|    critic_loss     | 199      |
|    learning_rate   | 0.001    |
|    n_updates       | 17612    |
------------------------

### Model 5: SAC

In [21]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 1000000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [22]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=80000)

Logging to tensorboard_log/sac/sac_1
----------------------------------
| environment/        |          |
|    portfolio_value  | 4.5e+06  |
|    total_cost       | 1.25e+05 |
|    total_reward     | 3.5e+06  |
|    total_reward_pct | 350      |
|    total_trades     | 64851    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 26       |
|    time_elapsed     | 380      |
|    total timesteps  | 10064    |
| train/              |          |
|    actor_loss       | 558      |
|    critic_loss      | 1.74e+03 |
|    ent_coef         | 0.108    |
|    ent_coef_loss    | -104     |
|    learning_rate    | 0.0001   |
|    n_updates        | 9963     |
----------------------------------
----------------------------------
| environment/        |          |
|    portfolio_value  | 3e+06    |
|    total_cost       | 9.44e+03 |
|    total_reward     | 2e+06    |
|    total_reward_pct | 200      |
|    total_trades     | 46290    |
| time/           

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [23]:
data_turbulence = processed_full[(processed_full.date<'2019-01-01') & (processed_full.date>='2009-01-01')]
insample_turbulence = data_turbulence.drop_duplicates(subset=['date'])

In [24]:
insample_turbulence.turbulence.describe()

count    2516.000000
mean       33.278198
std        33.999895
min         0.000000
25%        15.233636
50%        25.166759
75%        39.289961
max       332.851269
Name: turbulence, dtype: float64

In [25]:
turbulence_threshold = np.quantile(insample_turbulence.turbulence.values,1)

In [26]:
turbulence_threshold

332.85126857983164

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2018-12 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [27]:
trade = data_split(processed_full, '2019-01-01','2021-01-01')
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 380, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=model_sac, 
    environment = e_trade_gym)

hit end!


In [28]:
df_account_value.shape

(505, 2)

In [29]:
df_account_value.head()

,date,account_value
0,2019-01-02,1.000000e+06
1,2019-01-03,9.963386e+05
2,2019-01-04,1.006066e+06
3,2019-01-07,1.008263e+06
4,2019-01-08,1.017674e+06


In [30]:
df_actions.head()

,AAPL,AXP,BA,CAT,CSCO,CVX,DD,DIS,GS,HD,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PFE,PG,RTX,TRV,UNH,V,VZ,WBA,WMT,XOM
0,0,97,99,99,0,48,0,98,86,13,98,0,82,0,93,29,0,0,0,88,0,76,16,0,50,0,58,0,97,63
1,0,97,94,95,68,94,0,98,76,-13,88,0,82,0,67,96,0,0,0,80,0,-76,91,0,80,0,73,0,97,96
2,0,96,97,75,33,64,0,88,93,40,98,0,99,0,90,79,0,0,0,88,0,98,93,0,75,0,89,0,93,4
3,0,84,96,90,91,85,0,99,99,-9,97,0,90,0,38,93,0,0,0,70,0,-83,50,0,92,0,96,0,93,90
4,0,99,95,84,17,16,0,99,93,18,97,0,72,0,-50,90,0,0,0,26,0,-15,-23,0,79,0,62,0,83,22


<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [31]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.186750
Cumulative returns     0.409333
Annual volatility      0.225053
Sharpe ratio           0.875060
Calmar ratio           0.748224
Stability              0.324021
Max drawdown          -0.249591
Omega ratio            1.185554
Sortino ratio          1.270542
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.981014
Daily value at risk   -0.027573
dtype: float64


<a id='6.2'></a>
## 7.2 BackTestPlot

In [32]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = '2019-01-01',
             baseline_end = '2021-01-01')

==============Compare to DJIA===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (505, 8)


Start date,2019-01-02
End date,2020-12-31
Total months,24
,Backtest
Annual return,18.7%
Cumulative returns,40.9%
Annual volatility,22.5%
Sharpe ratio,0.88
Calmar ratio,0.75
Stability,0.32
Max drawdown,-25.0%


AttributeError: ignored

<a id='6.3'></a>
## 7.3 Baseline Stats

In [33]:
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", start = '2019-01-01',
                                  end = '2021-01-01')

stats = backtest_stats(baseline_df, value_col_name = 'close')


==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (505, 8)
Annual return          0.144674
Cumulative returns     0.310981
Annual volatility      0.274619
Sharpe ratio           0.631418
Calmar ratio           0.390102
Stability              0.116677
Max drawdown          -0.370862
Omega ratio            1.149365
Sortino ratio          0.870084
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.860710
Daily value at risk   -0.033911
dtype: float64
